## Purpose - To scrape data on authors, their affiliation and reported addresses at the level of each paper

### Import Packages

In [1]:
import pandas as pd 
import os 
import numpy as np 
import re
import string
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import math
from selenium.common.exceptions import NoSuchElementException
import pickle

In [2]:
EXE_PATH = r"C:\Users\nik596\Downloads\chromedriver_win32\chromedriver.exe"
browser = webdriver.Chrome(executable_path=EXE_PATH)

C:\Users\nik596\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [3]:
base = "https://www.nber.org/papers?page=1&perPage=100&sortBy=public_date"
browser.get(base)
browser.implicitly_wait(10)

text = browser.find_element_by_class_name("search__info-results").text
pages = int(text[text.find('F') + 2 : text.find('R') - 1]) + 1
pages = math.ceil(pages/100)
pages

C:\Users\nik596\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """


305

In [ ]:
paper_links = []
url = "https://www.nber.org/papers?page={q}&perPage=100&sortBy=public_date"

for i in range(1, pages + 1, 1):
    
    if i%10 ==0:
        print(i)
    
    browser.get(url.format(q = str(i)))
    browser.implicitly_wait(30)
    
    
    for element in browser.find_elements_by_xpath('//a[contains(@href,"/papers/")]'):
        paper_link = element.get_attribute('href')
        paper_links.append(paper_link)

In [ ]:
paper_links = set(paper_links)
with open(r'C:\Users\nik596\Documents\GitHub\NBER\papers.txt','wb') as f:
   pickle.dump(paper_links, f)

In [2]:
with open(r'C:\Users\nik596\Documents\GitHub\NBER\papers.txt','rb') as f:
   paper_links = pickle.load(f)

In [3]:
with open(r'C:\Users\nik596\Documents\GitHub\NBER\NBER_WP.txt','rb') as f:
   papers = pickle.load(f)
df=pd.DataFrame.from_dict(papers).T
#df = pd.read_excel(r'C:\Users\nik596\Dropbox (Harvard University)\New folder\NBER_WP.xlsx')
scraped = list(df['paper_link'])
len(scraped)
#scraped = []

3950

In [ ]:
dic = papers
count = len(papers)

errors = []

for paper_link in paper_links:
        
        try:
            html = urlopen(paper_link)
            soup = BeautifulSoup(html, 'lxml')
        
            paper_name = soup.find('h1', class_ = 'page-header__title').text.strip()
        
            if paper_link not in scraped:
                dic[count] = {'paper': paper_name}
                #print(paper_name)
                dic[count]['paper_link'] = paper_link
                
                paper_month = soup.find('time').text
                dic[count]['paper month'] = paper_month
                
                temp = soup.find('div', class_ = 'page-header__citation-item').text.strip()
                dic[count]['paper number'] = temp[temp.find('Paper') + 6:]
                
                for each in soup.find_all('div', class_ = 'info-grid__item'):
                    text = [t.text for t in each.find_all('a')]
                    l = len(text)
    
                    if 'topics' in each.text.lower():
                        for c in range(l):
                            dic[count]['paper topic ' + str(c + 1)] = text[c]
                    elif 'programs' in each.text.lower():
                        for c in range(l):
                            dic[count]['paper program ' + str(c + 1)] = text[c]
                    elif 'working groups' in each.text.lower():
                        for c in range(l):
                            dic[count]['paper group ' + str(c + 1)] = text[c]
                    elif 'projects' in each.text.lower():
                        for c in range(l):
                            dic[count]['paper project ' + str(c + 1)] = text[c]
        
                authors = soup.find_all('div', class_ = 'page-header__authors')[0].find_all('a')
        
                num = 1
                for author in authors:
            
                    author_name = author.text
                    author_link = author.get('href')
                    dic[count]['author ' + str(num)] = author_name
                    dic[count]['author link ' + str(num)] = author_link
            
                    author_link = "https://www.nber.org" + author_link
                    html = urlopen(author_link)
                    soup = BeautifulSoup(html, 'lxml')
            
                    author_position = soup.find_all('div', class_ = 'person-header__title')[0].text.strip()
                    dic[count]['author position ' + str(num)] = author_position
            
                    author_affiliation = soup.find_all('div', class_ = 'person-header__summary')[0].text.strip()
                    dic[count]['author affiliation ' + str(num)] = author_affiliation
                    
                    author_address = soup.find_all('div', 'contact-card__social')[-1].text.strip().replace('\n', ' ')
                    dic[count]['author address ' + str(num)] = author_address
            
                    num+=1
            
                count += 1
                if count%50 ==0:
                    print(count)
                    print(paper_name)
                    with open(r'C:\Users\nik596\Documents\GitHub\NBER\NBER_WP.txt','wb') as f:
                       pickle.dump(dic, f)
        except:
            errors.append(paper_link)

4000
Pledgeability, Industry Liquidity, and Financing Cycles
4050
Do Appeals to Donor Benefits Raise More Money than Appeals to Recipient Benefits? Evidence from a Natural Field Experiment with Pick.Click.Give.
4100
Evaluating the European View that the US has No Unemployment Problem
4150
Capital Taxation and Ownership when Markets are Incomplete
4200
Employee Stock Purchase Plans
4250
How Well do Foreign Exchange Markets Function: Might a Tobin Tax Help?
4300
Digitization and Pre-Purchase Information: The Causal and Welfare Impacts of Reviews and Crowd Ratings
4350
India's Lockdown: An Interim Report
4400
Purchasing-Power Annuities: Financial Innovation for Stable Real Retirement Income in an Inflationary Environment
4450
Do Enlarged Fiscal Deficits Cause Inflation: The Historical Record
4500
Commodity Price Volatility in the Biofuel Era: An Examination of the Linkage Between Energy and Agricultural Markets
4550
Short-run Effects of Parental Job Loss on Child Health
4600
The Long-run 

In [ ]:
df=pd.DataFrame.from_dict(dic).T
df.to_excel(r'C:\Users\nik596\Documents\GitHub\NBER\NBER_WP.xlsx',engine='xlsxwriter')

In [ ]:
df